In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN


smoten_noncontin = pd.read_csv('smoten_noncontin.csv', index_col=False)

pd.set_option("display.max_columns", None)


def ensembleScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('feature_names_in_')
    print(clf.feature_names_in_)
    print()
    print('feature_importances_')
    print(clf[1].feature_importances_)
    print()
    for i in range(len(clf.feature_names_in_)):
        print(clf.feature_names_in_[i], ":" ,clf[1].feature_importances_[i])
    print()
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print()
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print()
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print()
    print('\n')
    

print('Gradient Boost')

estimate = GradientBoostingClassifier(loss='log_loss', learning_rate=1, 
                                      n_estimators=150, subsample=1.0, 
                                      criterion='friedman_mse', min_samples_split=2, 
                                      min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                      max_depth=3, min_impurity_decrease=0.0, init=None, 
                                      random_state=None, max_features=None, verbose=0, 
                                      max_leaf_nodes=None, warm_start=False, 
                                      validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

print('### SMOTEN noncontin median impute ###')
print()

X = smoten_noncontin.drop('outcome',axis= 1)
y = smoten_noncontin['outcome']

print('Gradient Boost StandardScaler')
print()
ensembleScale_coef(StandardScaler(), estimate, 5, X, y)

print('Graident Boost RobustScaler')
ensembleScale_coef(RobustScaler(), estimate, 5, X, y)




Gradient Boost
### SMOTEN noncontin median impute ###

Gradient Boost StandardScaler

accuracy of each fold - [0.8030432309442548]
Avg accuracy : 0.16060864618885096
score 
 0.8044653014789533
feature_names_in_
['sofa_coagulation' 'sofa_cardiovascular' 'sofa_cns' 'sofa_renal' 'gender'
 'charlson_comorbidity_index']

feature_importances_
[0.08617161 0.10750339 0.54385649 0.05441017 0.03311227 0.17494606]

sofa_coagulation : 0.08617160693716779
sofa_cardiovascular : 0.10750339019258541
sofa_cns : 0.5438564938860564
sofa_renal : 0.054410172606935765
gender : 0.03311227340052283
charlson_comorbidity_index : 0.17494606297673168

fit
6 ['sofa_coagulation' 'sofa_cardiovascular' 'sofa_cns' 'sofa_renal' 'gender'
 'charlson_comorbidity_index']

classification report 
                   precision    recall  f1-score   support

Intubation False       0.81      0.79      0.80      7014
 Intubation True       0.80      0.81      0.81      7050

        accuracy                           0.80     140